In [1]:
# 基于平面波赝势方法的非局域赝势 (V_ps,nl) 示例实现
%run basis_func_with_kinetic.ipynb

import numpy as np
from dataclasses import dataclass
from typing import List, Sequence
from scipy.special import spherical_jn, sph_harm
import matplotlib.pyplot as plt

@dataclass
class GaussianProjectorChannel:
    """简单的高斯型投影子，近似常见 NCPP 中的 β_l(r)。"""
    l: int
    D_l: float  # 投影子强度系数 D_{I_s,l}
    r_cut: float  # (bohr)
    weight: float = 1.0

    def radial_beta(self, r: np.ndarray) -> np.ndarray:
        x = np.clip(r / self.r_cut, 0.0, None)
        base = (x ** self.l) * np.exp(-x**2)
        norm = np.sqrt(np.trapz(base**2 * r**2, r) + 1e-20)
        return self.weight * base / norm

def radial_grid(max_rcut: float, npts: int = 800, factor: float = 6.0) -> np.ndarray:
    r_max = max_rcut * factor
    grid = np.linspace(0.0, r_max, npts)
    grid[0] = 1e-6
    return grid

def compute_Q_lm(k_plus_G: np.ndarray, channel: GaussianProjectorChannel, r_grid: np.ndarray):
    """返回 shape=(nPW, 2l+1) 的 Q_{lm}(k+G) 数组。"""
    n_pw = k_plus_G.shape[0]
    l = channel.l
    beta_r = channel.radial_beta(r_grid)
    Q = np.zeros((n_pw, 2 * l + 1), dtype=complex)
    mags = np.linalg.norm(k_plus_G, axis=1)
    dirs = np.zeros_like(k_plus_G)
    nonzero = mags > 1e-10
    dirs[nonzero] = k_plus_G[nonzero] / mags[nonzero, None]

    for idx, _ in enumerate(k_plus_G):
        k_mag = mags[idx]
        jl = spherical_jn(l, k_mag * r_grid)
        radial = np.trapz(((-1j) ** l) * beta_r * jl * r_grid**2, r_grid)
        if k_mag < 1e-10:
            theta = 0.0
            phi = 0.0
        else:
            kz = dirs[idx, 2]
            theta = np.arccos(np.clip(kz, -1.0, 1.0))
            phi = np.arctan2(dirs[idx, 1], dirs[idx, 0])
        m_idx = 0
        for m in range(-l, l + 1):
            Y_lm = sph_harm(m, l, phi, theta)
            Q[idx, m_idx] = radial * Y_lm
            m_idx += 1
    return Q

def nonlocal_pseudopotential_matrix(
    a_vecs: np.ndarray,
    atom_positions: Sequence[np.ndarray],
    projectors: List[GaussianProjectorChannel],
    Ecut_eV: float,
    kvec: np.ndarray | None = None,
    r_pts: int = 800,
    Gvecs_override: np.ndarray | None = None,
 ):
    """构造给定 k 点和平面波基组上的 V_ps,nl 矩阵。"""
    b_vecs, Omega = reciprocal_vectors(a_vecs)
    if Gvecs_override is not None:
        Gvecs = np.array(Gvecs_override, copy=True)
    else:
        Gvecs, _, _ = generate_G_vectors(b_vecs, Ecut_eV, kvec=kvec)
    if Gvecs.size == 0:
        raise RuntimeError('未生成任何平面波，请提高 Ecut 或检查晶格。')
    if kvec is None:
        kvec = np.zeros(3)
    k_plus_G = Gvecs + kvec

    max_rc = max(ch.r_cut for ch in projectors)
    r_grid = radial_grid(max_rc, npts=r_pts)

    n_pw = len(Gvecs)
    Vnl = np.zeros((n_pw, n_pw), dtype=complex)
    prefac = (16.0 * np.pi**2) / Omega

    for channel in projectors:
        Q = compute_Q_lm(k_plus_G, channel, r_grid)  # shape (nPW, 2l+1)
        for m_idx in range(Q.shape[1]):
            q_vec = Q[:, m_idx]
            outer = np.outer(q_vec, np.conj(q_vec))
            Vnl += prefac * channel.D_l * outer

    if atom_positions:
        phase = np.zeros_like(Vnl)
        for tau in atom_positions:
            phase_vec = np.exp(-1j * Gvecs @ tau)
            phase += phase_vec[:, None] * np.conj(phase_vec[None, :])
        Vnl *= phase / len(atom_positions)

    return Gvecs, Vnl

Crystal: Al (FCC), a = 4.05 Å
Ecut = 300.0 eV -> 11.02480 Hartree
Number of plane waves (Gamma): 181
First 8 G coefficients: [(0, 0, 0), (1, 1, 1), (1, 0, 0), (0, 1, 0), (0, 0, 1), (0, 0, -1), (-1, 0, 0), (-1, -1, -1)]
First 8 kinetic energies (Hartree): [0.         1.01098166 1.01098166 1.01098166 1.01098166 1.01098166
 1.01098166 1.01098166]
